In [ ]:
!pip install wandb

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import wandb
from wandb.keras import (
   WandbMetricsLogger,
   WandbModelCheckpoint,
)

In [ ]:
run = wandb.init(project="17_Flowers")

In [ ]:
config = wandb.config

In [ ]:
wandb_callbacks = [
   WandbMetricsLogger(log_freq=5),
   WandbModelCheckpoint("models"),
]

In [ ]:

dataset_path = "/content/drive/MyDrive/Data/17Flowers/train"
width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip=True,
    brightness_range=(0.8,1.2),
    zoom_range=0.1,
    shear_range=0.1,
    rotation_range=10,
    validation_split=0.2
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size=(width,height),
    batch_size=32,
    subset='training'
)

val_data = idg.flow_from_directory(
    dataset_path,
    target_size=(width,height),
    batch_size=32,
    subset='validation'
)

In [ ]:
some_images=next(train_data)
X=some_images[0]
Y=some_images[1]

plt.figure(figsize=(16,16))
for i in range(32):
  plt.subplot(4,8,i+1)
  plt.imshow(X[i])


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(width,height,3),
    pooling='avg'
)

In [ ]:
base_model.summary()

In [ ]:
for layer in base_model.layers[0:-4]:
  layer.trainable = False

In [ ]:
model = tf.keras.models.Sequential([
    base_model,
    layers.Dropout(0.2),
    layers.Dense(17,activation='softmax')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

In [ ]:
base_model.summary()

In [ ]:
plt_model = model.fit(train_data, validation_data=val_data,callbacks=wandb_callbacks, epochs=20)

In [ ]:
model.save("17_Flowers_v2.h5")
